<a href="https://colab.research.google.com/github/ccasanoval/RLtests/blob/master/DoomV5a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DoomV5a

RL  = SB3 : PPO

ENV = VizDoom

.

Config VizDoom:

http://www.cs.put.poznan.pl/visualdoomai/tutorial.html#config

https://github.com/Farama-Foundation/ViZDoom/blob/master/scenarios/

.

    available_buttons = {

        ATTACK
        SPEED
        STRAFE

        MOVE_RIGHT
        MOVE_LEFT
        MOVE_BACKWARD
        MOVE_FORWARD
        TURN_RIGHT
        TURN_LEFT

        SELECT_WEAPON1
        SELECT_WEAPON2
        SELECT_WEAPON3
        SELECT_WEAPON4
        SELECT_WEAPON5
        SELECT_WEAPON6

        SELECT_NEXT_WEAPON
        SELECT_PREV_WEAPON

        LOOK_UP_DOWN_DELTA
        TURN_LEFT_RIGHT_DELTA
        MOVE_LEFT_RIGHT_DELTA
    }

.

In [4]:

!pip install vizdoom
!pip install stable_baselines3


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [13]:


from vizdoom import DoomGame
import numpy as np
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import cv2

class VizDoomGym(Env):
    def __init__(self, scenario, render):

        self.game = DoomGame()
        self.game.load_config(f"{scenario}.cfg")

        self.game.set_window_visible(render)
        self.game.init()

        self.observation_space = Box(
            low=0, high=255, shape=(100, 160, 1), dtype=np.uint8
        )
        self.number_of_actions = 8#len(self.game.get_available_buttons())
        self.action_space = Discrete(self.number_of_actions)


    def step(self, action):
        actions = np.identity(self.number_of_actions)
        reward = self.game.make_action(actions[action], tics = 4)

        if self.game.get_state():
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            info = self.game.get_state().game_variables[0]  # ammo
        else:
            state = np.zeros(self.observation_space.shape)
            info = 0

        info = {"info": info}
        terminated = self.game.is_episode_finished()
        truncated = self.game.is_player_dead()
        return state, reward, terminated, truncated, info

    def reset(self, seed=0):
        self.game.new_episode()
        state = self.game.get_state().screen_buffer

        if self.game.get_state():
            info = self.game.get_state().game_variables[0]  # ammo
        else:
            info = 0

        return (self.grayscale(state), {"ammo": info})

    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160, 100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100, 160, 1))
        return state

    def close(self):
        self.game.close()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:

from stable_baselines3.common.callbacks import BaseCallback

class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, verbose=1, name="?"):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.name = name

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = '{}_{}'.format(self.name, self.n_calls)
            self.model.save(model_path)
        return True


In [15]:

#scenario = "simpler_basic"
#scenario = "rocket_basic"
#scenario = "basic"
#scenario = "deadly_corridor"
scenario = "deathmatch"
#scenario = "defend_the_center"
#scenario = "my_way_home"
#scenario = "cig"
#scenario = "health_gathering"
#scenario = "multi"
#scenario = "multi_duel"
#scenario = "multi_deathmatch"
#scenario = "predict_position"
#scenario = "take_cover"

modelName = "Mod8"+scenario  # @param {type:"string"}
modelNew = True            # @param {type:"boolean"}
modelTrain = True          # @param {type:"boolean"}



In [16]:
from stable_baselines3 import PPO

#####################

callback = TrainAndLoggingCallback(check_freq=10000, name=modelName)
env = VizDoomGym(render=False, scenario=scenario)

if not modelNew:
    model = PPO.load(modelName, env=env)
else:
    model = PPO(
        "CnnPolicy",
        env,
        verbose=1,
        seed=0,
        #learning_rate=0.0001,
        #n_steps=4096
    )

# TRAIN
if modelTrain:
    model.learn(total_timesteps=400000, callback=callback)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 164      |
|    ep_rew_mean     | 2.25     |
| time/              |          |
|    fps             | 233      |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 152         |
|    ep_rew_mean          | 2.15        |
| time/                   |             |
|    fps                  | 201         |
|    iterations           | 2           |
|    time_elapsed         | 20          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013461687 |
|    clip_fraction        | 0.098       |
|    clip_range     

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

env = VizDoomGym(render=True, scenario=scenario)
model = PPO.load(modelName, env=env)
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
env.close()

print(f"mean_reward:{mean_reward:.2f}")
print(f"std_reward:{std_reward:.2f}")